In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os.path import join as opj

data_root = '/om2/user/rogerjin/data'
data_path = opj(data_root, 'data_files_new.npz')

In [3]:
import numpy as np

data = np.load(data_path)
data_keys = list(data.keys())
data_keys

['peak_selector',
 'atac_train_small',
 'atac_test_small',
 'rna_train',
 'rna_test',
 'rna_good_feats',
 'train_idx',
 'test_idx',
 'cell_type_train',
 'cell_type_test']

In [4]:
data['rna_train'].shape

(6897, 7445)

In [5]:
data['rna_test'].shape

(2300, 7445)

In [6]:
data['atac_train_small'].shape

(6897, 3808)

In [7]:
rna_train = data['rna_train']

In [8]:
rna_train[0]

array([-0.16375966, -0.16154247, -0.29548194, ..., -0.19634512,
       -0.4072624 , -0.15920115])

In [9]:
import numpy as np
print(np.sum(rna_train < 0))
np.sum(rna_train > 0)

46208208


5139957

In [10]:
import sys

sys.path.append('/om2/user/rogerjin/GANOLI/ganoli')

In [11]:
import torch
torch.cuda.is_available()

False

In [12]:
from GanoliDataset import GanoliDataset

dataset = GanoliDataset(rna_train)

In [13]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=5)

In [14]:
for x in loader:
    print(x)
    break

tensor([[-0.1638, -0.1615, -0.2955,  ..., -0.1963, -0.4073, -0.1592],
        [-0.1638, -0.1615, -0.2955,  ..., -0.1963, -0.4073, -0.1592],
        [-0.1638, -0.1615,  1.9634,  ..., -0.1963, -0.4073, -0.1592],
        [-0.1638, -0.1615, -0.2955,  ..., -0.1963,  2.0423, -0.1592],
        [-0.1638, -0.1615, -0.2955,  ..., -0.1963, -0.4073, -0.1592]],
       dtype=torch.float64)


In [26]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

model = Ridge(alpha=10**3)
model.fit(data['rna_train'], data['atac_train_small'])

Ridge(alpha=1000)

In [27]:
model.score(data['rna_test'], data['atac_test_small'])

-1.015301847493901